In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA, KernelPCA
from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD

import pandas as pd
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
# from sklearn import cross_validation
from keras import regularizers

import tensorflow as tf
from keras import metrics
from keras import backend as K

from utils import *

Using TensorFlow backend.


In [ ]:
# getdata()

### Clean data

In [3]:
time_percentage = 0.9
explained_variance = 0.9
df = pd.read_csv("rescuetime_data-ac-min.csv")
data_pd = Clean_DF(df)
data_pd.clean_data(time_percentage=time_percentage)
data_pd.clean_df = data_pd.clean_df.reset_index()

# print(data_pd.clean_df)
# print(data_pd.popular_apps)
# data_pd.get_pca(explained_variance=explained_variance)

In [ ]:
print("Dataset size:", data_pd.clean_df.shape,'\n')
print("Number of apps that consume", time_percentage*100, "% of all users time: ",len(data_pd.popular_apps), '\n')
print("Cleaned dataset columns:",'\n', data_pd.clean_df.columns.values, '\n')
print("Number of components that explain", explained_variance*100,"% of the data: ",data_pd.pca_data.shape[1], '\n')

In [ ]:
# t-SNE embedding, perplexity = 75
ptsne100 = TSNE(n_components=3, verbose=0, perplexity=30, n_iter=5000)
ptsne_results100 = ptsne100.fit_transform(pca_data)


## Kernel PCA with t-SNE

In [ ]:
data_np = np.array(data_pd.clean_df['Activity Vector'].tolist())
# PCA decomposition
sk_kPCA = KernelPCA(kernel="rbf", n_components=30)
kpca_data = sk_kPCA.fit_transform(data_np)
# print(pca_data)

In [ ]:
c = data_pd.clean_df['Productivity Score'][-2000:]

In [ ]:
# t-SNE embedding, perplexity = 25
ktsne100 = TSNE(n_components=3, verbose=0, perplexity=100, n_iter=20000)
ktsne_results100 = ktsne100.fit_transform(kpca_data[-2000:])

# t-SNE embedding, perplexity = 25
ktsne75 = TSNE(n_components=3, verbose=0, perplexity=75, n_iter=20000)
ktsne_results75 = ktsne75.fit_transform(kpca_data[-2000:])

# t-SNE embedding, perplexity = 25
ktsne30 = TSNE(n_components=3, verbose=0, perplexity=50, n_iter=20000)
ktsne_results30 = ktsne30.fit_transform(kpca_data[-2000:])

# t-SNE embedding, perplexity = 25
ktsne5 = TSNE(n_components=3, verbose=0, perplexity=5, n_iter=20000)
ktsne_results5 = ktsne5.fit_transform(kpca_data[-2000:])

In [ ]:
# x = ktsne_results100[:,0]
# y = ktsne_results100[:,1]
# z = ktsne_results100[:,2]

# trace1 = go.Scatter3d(
#     x=x,
#     y=y,
#     z=z,
#     mode='markers',
#     marker=dict(
#         size=12,
#         color=c,                # set color to an array/list of desired values
#         colorscale='RdYlGn',   # choose a colorscale
#         opacity=0.8
#     )
# )

# data = [trace1]
# layout = go.Layout(
#     margin=dict(
#         l=0,
#         r=0,
#         b=0,
#         t=0
#     )
# )
# fig = go.Figure(data=data, layout=layout)
# py.iplot(fig, filename='t-SNE Visualization')

## Autoencoder Embedding

In [ ]:
# this is the size of our encoded representations
encoding_dim = 32  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

input_size = len(data_pd.clean_df['Activity Vector'][0])
# print(input_size)
# this is our input placeholder
input_img = Input(shape=(input_size,))
# "encoded" is the encoded representation of the input
encoded = Dense(encoding_dim, activation='relu')(input_img)

# "decoded" is the lossy reconstruction of the input
decoded = Dense(input_size, activation='sigmoid')(encoded)

# this model maps an input to its reconstruction
autoencoder = Model(input_img, decoded)

In [ ]:
# this model maps an input to its encoded representation
encoder = Model(input_img, encoded)

In [ ]:
# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim,))
# retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer(encoded_input))

In [ ]:
autoencoder.compile(optimizer='adadelta', loss='mean_squared_error')

In [ ]:
data = np.matrix(data_pd.clean_df['Activity Vector'].tolist())
# print(data.shape)

# train, validate = cross_validation.train_test_split(data, train_size=0.8, test_size=0.2)

# # train = train.transpose()
# # validate = validate.transpose()
# print(train.shape)
# print(validate.shape)

# # train.reshape((input_size, len(train)))
# # validate.reshape((input_size, len(validate)))


# autoencoder.fit(x=train, y=train,
#                 verbose=0,
#                 nb_epoch=100,
#                 batch_size=256,
#                 shuffle=True,
#                 validation_data=(validate, validate))

In [ ]:
# # Encode data
# encoded_vecs = encoder.predict(data)
# t-SNE embedding
# tsne = TSNE(n_components=2, verbose=1, perplexity=5, n_iter=5000)
# tsne_results = tsne.fit_transform(encoded_vecs)

## Encoded with AE into 2 dimensions

### AutoEncoder, Perplexity = 50

## Variational AutoEncoder 2-D Embedding

In [ ]:
batch_size = 25
original_dim = input_size
latent_dim = 3
intermediate_dim = 15
epochs = 25
epsilon_std = 1.0

x = Input(batch_shape=(batch_size, original_dim))
h = Dense(intermediate_dim, activation='relu')(x)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)


def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim))
    return z_mean + K.exp(z_log_var / 2) * epsilon

# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# we instantiate these layers separately so as to reuse them later
decoder_h = Dense(intermediate_dim, activation='relu')
decoder_mean = Dense(original_dim, activation='sigmoid')
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)

def vae_loss(x, x_decoded_mean):
        xent_loss = original_dim * metrics.binary_crossentropy(x, x_decoded_mean)
        kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return xent_loss + kl_loss


vae = Model(x, x_decoded_mean)
vae.compile(optimizer='adam', loss=vae_loss)
vae.summary()

In [ ]:
#train, validate
print(data.shape)
vae.fit(x=data[:10000], y=data[:10000],
        shuffle=True,
        nb_epoch = epochs,
        verbose=1,
        batch_size=batch_size,
        validation_split=0.05)

In [ ]:
# print(pca_data.shape)

In [ ]:
# encoder, from inputs to latent space
encoder = Model(x, z_mean)

# generator, from latent space to reconstructed inputs
decoder_input = Input(shape=(latent_dim,))


_h_decoded = decoder_h(decoder_input)
_x_decoded_mean = decoder_mean(_h_decoded)
generator = Model(decoder_input, _x_decoded_mean)

x_test_encoded = encoder.predict(data[:10000], batch_size=batch_size)


c = data_pd.clean_df['Productivity Score']
x = x_test_encoded[:,0]
y = x_test_encoded[:,1]
z = x_test_encoded[:,2]

trace1 = go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=12,
        color=c,                # set color to an array/list of desired values
        colorscale='RdYlGn',   # choose a colorscale
        opacity=0.8
    )
)

data = [trace1]
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='t-SNE Visualization')